In [1]:
%load_ext autoreload
%autoreload 2
import json
import numpy as np

In [46]:
with open("_processed_data/train_entpair2scope.json") as f:
    foo = json.load(f)

In [47]:
len(foo)

4317

In [2]:
with open("data/nyt/rel2id.json") as f:
    rel2id = json.load(f)

In [27]:
id2rel = {}
for key, value in rel2id.iteritems():
    id2rel[value] = key

In [28]:
id2rel

{0: u'NA',
 1: u'/location/neighborhood/neighborhood_of',
 2: u'/location/fr_region/capital',
 3: u'/location/cn_province/capital',
 4: u'/location/in_state/administrative_capital',
 5: u'/base/locations/countries/states_provinces_within',
 6: u'/business/company/founders',
 7: u'/location/country/languages_spoken',
 8: u'/people/person/place_of_birth',
 9: u'/people/deceased_person/place_of_death',
 10: u'/location/it_region/capital',
 11: u'/people/family/members',
 12: u'/location/us_state/capital',
 13: u'/location/us_county/county_seat',
 14: u'/people/profession/people_with_this_profession',
 15: u'/location/br_state/capital',
 16: u'/location/in_state/legislative_capital',
 17: u'/sports/sports_team/location',
 18: u'/people/person/religion',
 19: u'/location/in_state/judicial_capital',
 20: u'/business/company_advisor/companies_advised',
 21: u'/people/family/country',
 22: u'/time/event/locations',
 23: u'/business/company/place_founded',
 24: u'/location/administrative_divisi

In [29]:
N_realtion = len(id2rel)

In [30]:
N_realtion

53

In [31]:
with open("test_result/nyt_pcnn_ave_pred.json") as f:
    test_result = json.load(f)

In [32]:
len(test_result)/N_realtion

324

In [33]:
np.isnan(test_result[0]["score"])

False

In [34]:
prev = ""
pair_orders = []
for entity_pair in test_result:
    if entity_pair["entpair"] != prev:
        pair_orders.append(entity_pair["entpair"])
        prev = entity_pair["entpair"]

In [35]:
pair_orders

[u'2545305419813343240#3388743576386104116',
 u'4064233096852543081#3548482198042959154',
 u'3403001434899206750#1728621965179183540',
 u'7914671991292107918#7190250404844244214',
 u'7641644476409151842#318162714922681854',
 u'8135354348049144612#9143112047311025885',
 u'8135354348049144612#8135354348049144635',
 u'7408333094570602969#5160555195885225007',
 u'2854866318377382424#3213221497913347449',
 u'5441264052731449062#2474809336493274964',
 u'5481788751956567736#7454974581558658460',
 u'666906927419004829#6855135297336999727',
 u'7408333094570602969#4300865682340879727',
 u'2545305419813343240#9087112906320368600',
 u'6248951997846450202#5040720939975238648',
 u'363880549302579308#271872469903409079',
 u'8365122593492097692#2226192830108615853',
 u'6945102933131023539#7124056637235340246',
 u'65085718151596571#7253968438333327077',
 u'8742389098892215582#5034723353012679030',
 u'1928963725039509733#7593265715567029820',
 u'7938683841055331818#5865601768814605565',
 u'9095402871749

In [36]:
relation_probs = {}
for entity_pair in test_result:
    key = entity_pair["entpair"]
    probs = relation_probs.get(key,np.zeros((N_realtion,)))
    if np.isnan(entity_pair["score"]):
        prob = 0
    else:
        prob = entity_pair["score"]
    probs[entity_pair["relation"]-1] = prob
    relation_probs[key] = probs

In [37]:
relation_probs

{u'1009097477307120705#3817968375471178220': array([0.02704549, 0.0013463 , 0.00113532, 0.00083795, 0.00130232,
        0.0049054 , 0.00112742, 0.0575588 , 0.02151253, 0.000848  ,
        0.00148751, 0.00112283, 0.00119679, 0.00110637, 0.00104686,
        0.00112662, 0.00159647, 0.00193338, 0.00098638, 0.00118135,
        0.00141722, 0.00119452, 0.00688381, 0.01654279, 0.0013166 ,
        0.00176044, 0.00111186, 0.07185246, 0.05933297, 0.00122627,
        0.00162567, 0.00106991, 0.00105937, 0.00140183, 0.0010783 ,
        0.09740487, 0.00196451, 0.00156881, 0.00107487, 0.00141285,
        0.00102425, 0.00105465, 0.00117843, 0.00099245, 0.0011644 ,
        0.00081745, 0.0210786 , 0.54447436, 0.01350024, 0.00605318,
        0.00107533, 0.00095076, 0.        ]),
 u'1075591531980690805#8633689459149300803': array([0.0344079 , 0.00155335, 0.00137941, 0.00112159, 0.00165431,
        0.00574416, 0.00150345, 0.05493226, 0.02336909, 0.00095158,
        0.001936  , 0.00154763, 0.00171185, 0.0015

In [38]:
pred_ralations = []
for key in pair_orders:
    pred_ralations.append(id2rel[np.argmax(relation_probs[key])])

In [39]:
with open("data/nyt/test.json") as f:
    test_input = json.load(f)

In [40]:
len(test_input)

365

In [41]:
output = []
for single_input, pred_ralation in zip(test_input, pred_ralations):
    to_append = (single_input["sentence"].encode('utf-8'), single_input["head"]["word"].encode('utf-8'), 
                 single_input["tail"]["word"].encode('utf-8'), pred_ralation, single_input["relation"].encode('utf-8'))
    output.append(to_append)

In [42]:
with open("test_result/common_output.tsv", "w") as f:
    for element in output:
        normal_str = [str(word) for word in element]
        f.write("\t".join(normal_str) + "\n")

In [43]:
len(output)

331

In [48]:
with open("data/nyt/train.json") as f:
    train_input = json.load(f)

In [64]:
relations = []
for single_input in train_input:
    relations.append(single_input["relation"].encode('utf-8'))

In [65]:
for single_input in test_input:
    relations.append(single_input["relation"].encode('utf-8'))

In [69]:
relations = list(set(relations))

In [70]:
relation_mapping = {} # heuristic to put "None" as 0
for i, relation in enumerate(relations):
    relation_mapping[relation] = i+1
relation_mapping[relations[-1]] = 0

In [71]:
relation_mapping

{'/business/company/advisors': 15,
 '/business/company/founders': 1,
 '/business/company/major_shareholders': 18,
 '/business/company/place_founded': 9,
 '/business/company_shareholder/major_shareholder_of': 4,
 '/business/person/company': 17,
 '/location/administrative_division/country': 10,
 '/location/country/administrative_divisions': 21,
 '/location/country/capital': 19,
 '/location/location/contains': 20,
 '/location/neighborhood/neighborhood_of': 6,
 '/people/deceased_person/place_of_death': 3,
 '/people/ethnicity/geographic_distribution': 16,
 '/people/ethnicity/people': 5,
 '/people/person/children': 0,
 '/people/person/nationality': 13,
 '/people/person/place_lived': 12,
 '/people/person/place_of_birth': 2,
 '/people/person/religion': 8,
 '/sports/sports_team/location': 7,
 '/sports/sports_team_location/teams': 14,
 'None': 11}

In [73]:
with open('data/nyt/rel2id.json', 'w') as outfile:
    json.dump(relation_mapping, outfile)